Acerca del conjunto de datos
Contexto
En un documento publicado a principios de 2019, la previsión en los mercados energéticos se identifica como una de las áreas de mayor contribución de apalancamiento del aprendizaje automático/profundo hacia la transición a una infraestructura eléctrica basada en energías renovables.

Contenido
Este conjunto de datos contiene 4 años de datos de consumo eléctrico, generación, precios y clima para España. Los datos de consumo y generación se obtuvieron de la REGRT de Transporte, un portal público de datos de los operadores de servicios de transmisión (TSO). Los precios de liquidación se obtuvieron del TSO español Red Electric España. Los datos meteorológicos se compraron como parte de un proyecto personal de la API Open Weather para las 5 ciudades más grandes de España y se hicieron públicos aquí.

Agradecimientos
Estos datos están disponibles públicamente a través de ENTSOE y REE y se pueden encontrar en los enlaces anteriores.

Inspiración
El conjunto de datos es único porque contiene datos horarios de consumo eléctrico y las respectivas previsiones del GRT sobre el consumo y los precios. Esto permite comparar los pronósticos prospectivos con los pronósticos actuales del estado del arte que se utilizan en la industria.

Visualice las curvas de carga y oferta marginal.
¿Qué mediciones meteorológicas y ciudades influyen más en la demanda eléctrica, los precios, la capacidad de generación?
¿Podemos prever con 24 horas de antelación mejor que el TSO?
¿Podemos predecir el precio de la electricidad por hora del día mejor que el TSO?
Pronostique el precio intradía o la demanda eléctrica hora a hora.
¿Cuál es la fuente de próxima generación que se activará en la curva de carga?